In [2]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
google_api_key=os.getenv('GOOGLE_API_KEY') 


In [4]:
from llama_index.core import Settings,StorageContext,load_index_from_storage,VectorStoreIndex,SimpleDirectoryReader


In [5]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
import google.generativeai as genai


c:\Users\Shreyas\Desktop\SmartHire\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
genai.configure(api_key=google_api_key) 


In [7]:
doc=SimpleDirectoryReader(r"C:\Users\Shreyas\Desktop\SmartHire\experiments\data").load_data()


In [8]:
model=Gemini(models='gemini-1.5-pro',api_key=google_api_key)
gemini_embed_model=GeminiEmbedding(model_name='models/text-embedding-004')


In [9]:
Settings.llm = model 
Settings.embed_model=gemini_embed_model
Settings.chunk_size=800
Settings.chunk_overlap=20



In [10]:
index=VectorStoreIndex.from_documents(doc)
index.storage_context.persist()



In [11]:
parameter="Knowledge in ML" 
prompt= f"""You are an AI system designed to extract only the most relevant sections from a resume based on a given parameter. 
Focus on details from Work Experience, Skills, Education, Certifications, Projects, or Achievements 

Ignore personal details, details about education, unrelated experience, and generic skills.
Do not summarize the document, display it as it is.

"""





In [12]:
retriever = index.as_query_engine()
answer=retriever.query(prompt)
print(answer.response)




Software Developer Intern, CodeWorks Inc., Summer 2023 June 2022 – Present 
● Developed backend services using Java and Spring Boot 
● Collaborated on database design and SQL query optimization 
● Integrated REST APIs for web applications 

Programming Languages: Java, Python, C++  
Web Development: HTML, CSS, JavaScript (ES6)  
Frameworks/Technologies: Spring Boot, Flask, Docker, Kubernetes 
Database Management: SQL, PostgreSQL  
Version Control: Git, GitHub 

Distributed Chat Application: 
• Developed a real-time chat application using Java, WebSocket, and Spring Boot. 
• Implemented a scalable server architecture supporting thousands of concurrent users. 
• Integrated a NoSQL database (MongoDB) for persistent message storage. 
Smart Scheduler: 
• Built a web-based scheduling application using Python (Flask) and JavaScript. 
• Utilized machine learning algorithms (scikit-learn) to predict optimal task timings. 
• Designed a responsive user interface with HTML, CSS, and Bootstrap. 
Po

In [13]:
prompt1 = f"""
    You are an AI system designed to assist in resume screening. 
    Extract all the technical skills, technologies, programming languages, frameworks, 
    tools, and libraries from the given description. 

    The list should be very exhaustive and should not miss any skills, neither should it include any 
    skills which are not present in the description.


Skills Description:
{answer.response}

Output format: Comma separated 
Skill1,Skill2,Skill3, ...,SkillN """


In [14]:
response = model.complete(prompt1)
skill_list = [skill.strip() for skill in response.text.split(",")]



In [21]:
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

from sentence_transformers import SentenceTransformer
import numpy as np

# # Load a pre-trained sentence transformer model
# sentence_transformer_model = SentenceTransformer('BAAI/bge-large-en-v1.5')

# # Convert "Machine Learning" and each skill to embeddings
# target_skill = "Machine Learning"
# target_embedding = sentence_transformer_model.encode([target_skill])[0]

# # Generate embeddings for each skill and calculate similarity
# similarities = []
# for skill in skill_list:
#     skill_embedding = sentence_transformer_model.encode([skill])[0]
#     similarity = np.dot(target_embedding, skill_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(skill_embedding))
#     similarities.append((skill, similarity))

# # Sort skills by similarity score and print
# similarities.sort(key=lambda x: x[1], reverse=True)
# print("Similarity scores with 'Machine Learning':")
# for skill, score in similarities:
#     print(f"{skill}: {score:.4f}")




In [22]:
# Define the skills to compare
skills_to_compare = ["Huggingface", "Mongoose"]

# Generate embeddings for the target skills
target_embeddings = sentence_transformer_model.encode(skills_to_compare)

# Calculate similarity with "Machine Learning"
ml_embedding = sentence_transformer_model.encode(["Machine Learning"])[0]

print("Similarity scores with 'Machine Learning':")
for skill, embedding in zip(skills_to_compare, target_embeddings):
    similarity = np.dot(ml_embedding, embedding) / (np.linalg.norm(ml_embedding) * np.linalg.norm(embedding))
    print(f"{skill}: {similarity:.4f}")


Similarity scores with 'Machine Learning':
Huggingface: 0.5325
Mongoose: 0.6081



Ranked Skills:
PyTorch: 0.1164
TensorFlow: 0.1149
C++: 0.1121
Python: 0.1105
LangChain: 0.1104
React: 0.1103
SQL: 0.1100
: 0.1080
Huggingface: 0.1075
